In [ ]:
# libraries
import mne
import glob
%matplotlib widget
from os import chdir, makedirs
from os.path import join, basename, dirname
import matplotlib.pyplot as plt ## for basic plotting
import matplotlib as mpl ## for setting default parameters
import pandas as pd
from tqdm import tqdm

In [ ]:
# paths
MEG_dir = "/work/MEG_data/workshop_data"
subjects_dir = "/work/freesurfer"
behaviour_path = "/work/MEG_data/workshop_data/behavioural_logs"

# get MEG data filepaths for all subjects
MEG_files = sorted(glob.glob(f"{MEG_dir}/*/*/workshop_2025_raw.fif"))
MEG_files

['/work/MEG_data/workshop_data/0163/20250922_000000/workshop_2025_raw.fif',
 '/work/MEG_data/workshop_data/0164/20251003_000000/workshop_2025_raw.fif',
 '/work/MEG_data/workshop_data/0165/20250923_000000/workshop_2025_raw.fif',
 '/work/MEG_data/workshop_data/0166/20250923_000000/workshop_2025_raw.fif',
 '/work/MEG_data/workshop_data/0167/20250922_000000/workshop_2025_raw.fif',
 '/work/MEG_data/workshop_data/0168/20250924_000000/workshop_2025_raw.fif',
 '/work/MEG_data/workshop_data/0169/20250923_000000/workshop_2025_raw.fif',
 '/work/MEG_data/workshop_data/0170/20250924_000000/workshop_2025_raw.fif']

In [ ]:
mne.set_log_level('WARNING') # turning off verbose so loadbar is updated in-place

# for each subject (THIS TAKES APPROX. 11 MIN ON 8 CPU)
for MEG_file in tqdm(MEG_files):
    subject = basename(dirname(dirname(MEG_file))) # extract subject number from filepath
    print(f"Processing subject {subject}")

    # load MEG data
    raw = mne.io.read_raw_fif(MEG_file, preload=True) 

    # load behavioral data
    behaviour_file = glob.glob(f"{behaviour_path}/{subject}*_experiment_data.csv") # match file that starts with subject number
    if behaviour_file:
        behaviour = pd.read_csv(behaviour_file[0], index_col=False)

    # error handling
    else:
        print("No matching behaviour file found")

    # filter by frequency
    raw.filter(h_freq=40, l_freq=1) # band pass
    #fig = raw.plot()

    # find events
    events = mne.find_events(raw, min_duration=0.002)

    # identify event indices where stimulus is shown (event code 1 and 3)
    target_indices = events[ : , 2] < 4

    # rename those events by a code indicating their following PAS score
    events[target_indices, 2] = behaviour['subjective_response'] + 20
    #behaviour['subjective_response'].value_counts()

    # checking which event types (PAS responses) are present
    present = set(events[:, 2])
    wanted = {21, 22, 23, 24}
    missing = wanted - present

    # prepare dict with event names
    if not missing:
        event_id = dict(NE=21, WG=22, ACE=23, CE=24)
        print("All PAS responses present.")

    # dealing with participants who never used PAS 4
    elif missing == {24}:
        event_id = dict(NE=21, WG=22, ACE=23)
        print("Missing PAS 4 response. Only using PAS 1-3.")

    # error handling
    else:
        print(f"Missing unexpected event codes: {missing}")

    # creating epochs
    epochs = mne.Epochs(raw,
                        events,
                        event_id = event_id,
                        tmin = -0.200,
                        tmax = 0.550,
                        baseline = (-0.200, 0),
                        reject = dict(mag=4e-12, grad=4e-10, eog=250e-6), # rejecting criteria
                        preload = True) # telling it to apply the rejection
    #epochs.plot(block=True)

    # save epochs
    makedirs('epochs', exist_ok=True) # making sure folder exists
    epochs.save(f'epochs/{subject}-epo.fif', overwrite=True)

    # compute forward model
    info = epochs.info
    trans = glob.glob(f"{MEG_dir}/{subject}/*/workshop_2025-trans.fif")[0]
    src = join(subjects_dir, subject, 'bem', subject + '-oct-6-src.fif')
    bem = join(subjects_dir, subject, 'bem', subject + '-5120-bem-sol.fif')

    fwd = mne.make_forward_solution(
                                info, trans,
                                src, bem)

    # save forward model
    makedirs('models', exist_ok=True) # making sure folder exists
    mne.write_forward_solution(f'models/{subject}-fwd.fif', fwd, overwrite=True)

  0%|          | 0/8 [00:00<?, ?it/s]

Processing subject 0163
Missing PAS 4 response. Only using PAS 1-3.


 12%|█▎        | 1/8 [01:15<08:45, 75.05s/it]

Processing subject 0164
All PAS responses present.


 25%|██▌       | 2/8 [02:30<07:31, 75.23s/it]

...

 75%|███████▌  | 6/8 [07:46<02:35, 77.94s/it]

Processing subject 0169
All PAS responses present.


 88%|████████▊ | 7/8 [09:10<01:19, 79.90s/it]

Processing subject 0170
All PAS responses present.


100%|██████████| 8/8 [10:29<00:00, 78.63s/it]
